In [2]:
import numpy as np
import pandas as pd

In [3]:
class Node():
    
    def __init__(self, node_type = 'Node', side=None, feature = None, split_value = None, parent=None, *, node_depth = None,  ID = None, value = None, children=[]): #class initialization
        self.node_type = node_type #Leaf or Node        
        self.side = side #LEft or Right
        self.feature = feature
        self.split_value = split_value
        self.parent = parent
        
        self.node_depth = node_depth 
        self.ID = ID
        self.value = value #only for Leaf  
        self.children = []
        
        self.Np = 0 #number of points in the node
        
        self.leaf_targets_index = None
        
        
        
            
    def set_value(self, new_value):
        self.value = new_value
        
    def add_child(self, new_value):
        self.children.append(new_value)
        
class MyTreeReg():
    
    def __init__(self, max_depth = 5,  min_samples_split =2, max_leafs = 20, bins = None): 
        self.max_depth = max_depth #maximum possible depth of tree
        self.min_samples_split = min_samples_split #minimum sample split
        self.max_leafs = max_leafs #maximum possible number of leaves in a tree
        self.bins = bins
        
        #tree parameters
        self.tree = []
        self.leafs_cnt = 0 #number of created leaves in the tree
        self.potential_leafs_cnt = 1 #counting potential leaves
        self.leafs_sum = 0 #sum of the leaves values
        
        self.histogram = {}
        
        self.fi = {}
        
        self.N = 0
        
    def set_N(self, new_value):
        self.N = new_value
        
    def __repr__(self):
        return f'MyTreeReg class: max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}'
    
    #------------------------------------------------------------------------------------------------------------
    def fit(self, X, y): #receives panda dataframe and series
        if self.bins != None:
            for feature in X.columns:
                self.histogram.update({feature: self.get_hist_delimeters(X[feature].values)})
                
        for feature in X.columns:
            self.fi.update({feature: 0})
        
        #Create root node
        feature, split_value, ig = self.get_best_split(X,y)
        X_left, y_left, X_right, y_right, y_left_idx, y_right_idx = self.split_dataframe(X, y, feature, split_value)
        if ig == 0.0 or len(y_left) == 0 or len(y_right) == 0:            
            print('All targets belong to class:', np.sum(y.values)/len(y.values) )
        else:
            _node = self.register_Node("Node", None, feature, split_value, None, y, None)
            #--------feature importance update---
            _node.Np = len(y.values)
            self.update_fi(y, _node, None)
            #----------------------------------
            self.grow_tree(X_left, y_left, 'Left', _node, y_left_idx)
            self.grow_tree(X_right, y_right, 'Right', _node, y_right_idx)
            
        
    def grow_tree(self, X, y, side, parent, y_idx): #receives panda dataframe and series, string and Node
        feature, split_value, ig = self.get_best_split(X,y)
        X_left, y_left, X_right, y_right,y_left_idx, y_right_idx = self.split_dataframe(X, y, feature, split_value)
        
        if ig != 0.0 and len(y_left) != 0 and len(y_right) != 0 and (parent.node_depth < self.max_depth) and (len(y.values) >= self.min_samples_split) and (self.leafs_cnt + self.potential_leafs_cnt < self.max_leafs):
            _node = self.register_Node('Node', side, feature, split_value, parent, y, y_idx)
            #--------feature importance update---
            _node.Np = len(y.values)
            self.update_fi(y, _node, parent)
            #----------------------------------
            self.grow_tree(X_left, y_left, 'Left', _node, y_left_idx)
            self.grow_tree(X_right, y_right, 'Right', _node, y_right_idx)
        else:
            _node = self.register_Node('Leaf', side, feature, split_value, parent, y, y_idx)
            #--------feature importance update---
            _node.Np = len(y.values)
            self.update_fi(y, _node, parent)
            #----------------------------------
            return _node
            
    #-------------------------------------------------------------------------------------------------------------
    #-------------------------------------------------------------------------------------------------------------
    def register_Node(self, node_type, side, feature, split_value, parent, y, y_idx):
        #1.setting node depth
        if parent != None:
            node_depth = parent.node_depth +1
        else:
            node_depth = 1
            
        #2.setting node ID
        if node_type == 'Node':
            if side == 'Left':
                 ID = parent.ID + '.1' 
            elif side == 'Right':
                 ID = parent.ID + '.2'
            else:
                ID = '1'
        
        if node_type == 'Leaf':
            ID = parent.ID
            
        #3.Setting node value
        if node_type == 'Leaf':
            value = np.mean(y.values)
        else:
            value = None
        
        new_node = Node(node_type, side, feature, split_value, parent, node_depth=node_depth, ID=ID, value=value)
        new_node.leaf_targets_index = y_idx
        self.tree.append(new_node)
        
        #add as a child to parent node
        if parent != None :                            
            parent.add_child(new_node)
            
        #update counts
        if node_type == "Node":
            self.potential_leafs_cnt = self.potential_leafs_cnt + 1
        elif node_type == "Leaf":
            self.leafs_cnt = self.leafs_cnt + 1
            self.potential_leafs_cnt = self.potential_leafs_cnt - 1
            self.leafs_sum = self.leafs_sum + value
        return new_node
        
            
    def print_tree_full(self):
        for node in self.tree:
                print(node.__dict__)
    
    def print_tree(self):
        for node in self.tree:
            if node.node_type == 'Node':
                
                print(node.__dict__['ID'], node.__dict__['feature'], '>', node.__dict__['split_value'])
            else:
                print(node.__dict__['ID'],node.__dict__['side'], '-', node.__dict__['value'])
                
    def move_up_the_tree(self, X, _node,i):
        if _node.node_type =='Leaf':
            self.predictions[i]=float(_node.value)
        elif _node.node_type =='Node': 
            if X[_node.feature] <= _node.split_value:
                if _node.children[0].side =='Left':
                    self.move_up_the_tree(X, _node.children[0],i)
                else: self.move_up_the_tree(X, _node.children[1],i)
               
            else:
                if _node.children[0].side =='Right':
                    self.move_up_the_tree(X, _node.children[0],i)
                else: self.move_up_the_tree(X, _node.children[1],i)
    #-------------------------------------------------------------------------------------------------------------
    #-------------------------------------------------------------------------------------------------------------
    def predict(self,X):
        self.predictions = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            self.move_up_the_tree(X.iloc[i,:], self.tree[0],i)
        return(self.predictions)
        
    #-------------------------------------------------------------------------------------------------------------
    #-------------------------------------------------------------------------------------------------------------
    #Calculate the best split
    def MSE(self, y): # receives 1D numpy array
        return np.mean(np.square(y-np.mean(y)))

    def data_split(self, X, y, threshold): #receives two 1D numpy arrays and a float
        X_left = X[X <= threshold]
        y_left = y[X <= threshold]
        X_right = X[X > threshold]
        y_right = y[X > threshold]
        return X_left, y_left, X_right, y_right
    
    def get_IG(self, X, y, threshold): #receives two numpy arrays and a float
        #split the data by the threshold
        _, y_left, __, y_right = self.data_split(X, y, threshold)
        if len(y_left) == 0 or len(y_right) == 0: #threshold does not split the data
            return 0.0
        else:
            S0 = self.MSE(y)
            S1 = self.MSE(y_left)*len(y_left)/len(y)
            S2 = self.MSE(y_right)*len(y_right)/len(y)
            IG = S0 - S1 -S2
            return IG
        
    def get_native_delimeters(self, X): #receives 1D numpy array
        X_unique = np.unique(np.sort(X))
        native_delimeters = [np.mean([X_unique[i-1], X_unique[i]]) for i in range(1, len(X_unique))]
        return native_delimeters
    
    def get_hist_delimeters(self, X): #receives 1D numpy array
        hist_delimeters = np.histogram(X, self.bins)[1][1:-1]
        return hist_delimeters
    
    def get_best_split(self, X, y): #receives panda dataframe and panda series
        feature_best_split = {}
        for feature in X.columns:
            if len(X[feature].values) == 0 or np.max(X[feature].values) == np.min(X[feature].values):
                feature_best_split.update({feature: [None, 0.0]}) #feature has no values or any delimeters
            else:
                if self.bins == None:                    
                    feature_delimeters = self.get_native_delimeters(X[feature].values)
                else: 
                    X_unique = np.unique(np.sort(X))
                    if len(X_unique) <= self.bins:
                        feature_delimeters = self.get_native_delimeters(X[feature].values)
                    else:
                         feature_delimeters = self.histogram[feature]
                    
                feature_igs = [self.get_IG(X[feature].values, y.values, feature_delimeters[i]) for i in range(len(feature_delimeters))]
                feature_best_split.update({feature: [feature_delimeters[np.argmax(feature_igs)],np.max(feature_igs)]})
        
        split_value, ig   = max(feature_best_split.values(), key=lambda x: x[1])
        feature = next(k for k, v in feature_best_split.items() if v == [split_value, ig])
        return feature, split_value, ig
    #-------------------------------------------------------------------------------------------------------------
        
    def split_dataframe(self, X, y, feature, threshold): #X,y - np.arrays, threshold - float
        X_left = X[X[feature] <= threshold]
        y_left = y[X[feature] <= threshold]
        X_right = X[X[feature] > threshold]
        y_right = y[X[feature] > threshold]
        return X_left, y_left.drop(columns='index'), X_right, y_right.drop(columns='index'), y_left.index.values, y_right.index.values
    
    def update_fi(self, y, _node, parent):
        if _node.node_type == 'Node':
            FI = self.MSE(y.values)*_node.Np/self.N         
            self.fi.update({_node.feature: self.fi[_node.feature] + FI })
        
        if parent != None:
            FI = self.MSE(y.values)*_node.Np/self.N
            self.fi.update({parent.feature: self.fi[parent.feature] - FI })

In [4]:
class MyBoostClf():
    def __init__(self, n_estimators=10, learning_rate = 0.1, max_depth = 5, min_samples_split = 2, max_leafs = 20, bins =16, metric = None, max_features = 0.5, max_samples = 0.5, random_state = 42, reg = 0.1):
        self.n_estimators= n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.max_leafs = max_leafs
        self.bins = bins
        
        self.pred_0 = 0
        self.trees = []
        
        self.eps = 1e-15 
        
        self.metric = metric
        self.best_score = None
        
        self.max_features = max_features
        self.max_samples = max_samples
        self.random_state = random_state
        
        self.reg = reg
        self.trees_leafs_cnt=0
        
        self.fi = {}
        
    
    def __repr__(self):
        return f'MyBoostClf class: n_estimators={self.n_estimators}, learning_rate={self.learning_rate}, max_depth={self.max_depth}, min_samples_split={self.min_samples_split}, max_leafs={self.max_leafs}, bins={self.bins}'
    
    def fit(self, X, y, X_eval, y_eval, early_stopping, verbose=None):
        for feature in X.columns:
            self.fi.update({feature: 0.0})
        
        random.seed(self.random_state)
        p = np.mean(y)
        log_odds = np.log(p/(1-p+self.eps))
        self.pred_0 = log_odds
        
        predictions = np.ones(y.shape[0])*self.pred_0 #predictions expressed as log(odds)
        
        if early_stopping != None and not X_eval.empty and not y_eval.empty:
            predictions_eval = np.ones(y_eval.shape[0])*self.pred_0
            if self.metric != None:
                self.best_score = 0
            else:
                self.best_score = float('inf')
            cnt = 0
            k=0
            
        
        for i in range(self.n_estimators):
            prob = self.log_odds_to_proba(predictions)
            antiG = y - prob
            
            tree = MyTreeReg(self.max_depth, self.min_samples_split, self.max_leafs, self.bins)
            tree.set_N(y.shape[0])
            
            cols_idx = random.sample(list(X.columns), int(np.round(self.max_features*len(X.columns),0))) 
            rows_idx = random.sample(range(X[cols_idx].shape[0]), int(np.round(self.max_samples*X[cols_idx].shape[0],0))) #select samples
            tree.fit(X.loc[rows_idx,cols_idx],antiG[rows_idx])
            
            
            for node in tree.tree:
                if node.node_type == 'Leaf':
                    y_ = y.loc[node.leaf_targets_index].values #initial targets
                    pred = self.log_odds_to_proba(predictions[node.leaf_targets_index]) #predictions expressed as log_odds
                    node.set_value(self.get_gamma(y_, pred) + self.reg*self.trees_leafs_cnt)
            
            self.trees.append(tree)
            self.trees_leafs_cnt = self.trees_leafs_cnt + tree.leafs_cnt
           
            predictions = predictions + self.get_learning_rate(i)*tree.predict(X)
           
            
            if early_stopping != None and not X_eval.empty and not y_eval.empty:    
                
                    predictions_eval = predictions_eval + self.get_learning_rate(i)*tree.predict(X_eval)
                    predictions_eval_proba = self.log_odds_to_proba(predictions_eval)
                    predictions_eval_pred =  (predictions_eval_proba > 0.5)*1
                    score_eval = self.calculate_metric(y_eval, predictions_eval_pred, predictions_eval_proba)
                    if self.metric != None:
                        if score_eval > self.best_score:
                            cnt = 0
                            self.best_score = score_eval
                        else:
                            cnt += 1
                    else:
                        if score_eval < self.best_score:
                            cnt = 0
                            self.best_score = score_eval
                        else:
                            cnt += 1
                    
                    if cnt == early_stopping:
                        return None
                    else:
                        k += 1
                        
                            
                            
                        
                    
 
        
        
        
        
        self.best_score = self.calculate_metric(y, self.predict(X), self.predict_proba(X))
        
        self.update_fi()
            
            
            
    def log_odds_to_proba(self, x):
        return np.exp(x)/(1+np.exp(x))
    
    def get_gamma(self, y_, p):
        return np.sum(y_-p)/np.sum(p*(1-p))
    
    def get_loss(self, y_,probs):
        return -np.mean(y_*np.log(probs+self.eps) + (1-y_)*np.log(1-probs))
        
    def predict_proba(self, X):
        eps = 1e-15 
        predictions = np.ones(X.shape[0])*self.pred_0
        k = 0
        for tree in self.trees:
            predictions = predictions + self.get_learning_rate(k)*tree.predict(X)
            k += 1
        return np.exp(predictions)/(1+np.exp(predictions))
    
    def predict(self, X):
        return (self.predict_proba(X) > 0.5)*1
    
    def calculate_metric(self,y_, pred, probs, beta = 1):
        TP=np.count_nonzero((y_==1)&(pred==1))
        TN = np.count_nonzero((y_==0)&(pred==0))
        FP = np.count_nonzero((y_==0)&(pred==1))
        FN = np.count_nonzero((y_==1)&(pred==0))
        
        if self.metric == 'accuracy':
            return (TP+TN)/(TP+TN+FP+FN)
        elif self.metric == 'precision':
            return TP/(TP+FP)
        elif self.metric == 'recall':
            return TP/(TP+FN)
        elif self.metric == 'f1':
            pres = TP/(TP+FP)
            rec = TP/(TP+FN)
            return (1+np.square(beta))*pres*rec/(np.square(beta)*pres + rec)
        elif self.metric  == 'roc_auc':
            probs = np.round(probs,10)
            sorted_idx = np.argsort(-probs)
            probs_sorted = probs[sorted_idx]
            y_sorted = y_[sorted_idx]
            
            sum=0.
            P = len(np.where(y_sorted==1)[0])
            N = len(np.where(y_sorted==0)[0])
            
            
            for prob, class_ in zip(probs_sorted,y_sorted):
                if class_ == 0.:
                    sum = sum + len(np.where(y_sorted[probs_sorted > prob]==1)[0])
                    sum = sum + 0.5*len(np.where(y_sorted[probs_sorted == prob]==1)[0])
            return  sum/(P*N)
        else:
            return self.get_loss(y_, probs)
            
    def get_learning_rate(self, i):
        if isinstance(self.learning_rate, float)== True:
            return self.learning_rate 
        else:
            return self.learning_rate(i+1)
        
    def update_fi(self):
        for tree in self.trees:
            for key in tree.fi.keys():
                self.fi.update({key: self.fi[key] + tree.fi[key] })